### Initialize Data 

In [1]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
import keras
 
input_shape = (32, 32, 3)
img_width = 32
img_height = 32
num_classes = 10
nb_train_samples = 1000
nb_validation_samples = 200
batch_size = 16
epochs = 1
 
train_data_dir = './number/train'
validation_data_dir = './number/test'
 
# Creating data generator for test data
validation_datagen = ImageDataGenerator(
    # used to rescale the pixel values from [0, 255] to [0, 1] interval
    rescale = 1./255)
 
# Creating data generator for training data
train_datagen = ImageDataGenerator(
      rescale = 1./255,             
      rotation_range = 10,          
      width_shift_range = 0.25,       
      height_shift_range = 0.25,      
      shear_range=0.5,
      zoom_range=0.5,
      horizontal_flip = False,        
      fill_mode = 'nearest')         
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        class_mode = 'categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        class_mode = 'categorical',
        shuffle = False)    

Using TensorFlow backend.
D:\Anaconda\ana\envs\opencv-env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\ana\envs\opencv-env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\ana\envs\opencv-env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\ana\envs\opencv-env\lib\site-packages\tensorflo

Found 1000 images belonging to 10 classes.
Found 4000 images belonging to 10 classes.


### Implement LeNet Model

In [2]:
# create model
model = Sequential()
 
# 2 sets of Convolution, RELU, Pooling
model.add(Conv2D(20, (5, 5),
          padding = "same", 
          input_shape = input_shape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
 
model.add(Conv2D(50, (5, 5),
          padding = "same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
 
# Fully connected layers 
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("relu"))
 
# Softmax 
model.add(Dense(num_classes))
model.add(Activation("softmax"))
           
model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.Adadelta(),
              metrics = ['accuracy'])
    
print(model.summary())

W0924 09:11:33.509381 24064 deprecation_wrapper.py:119] From D:\Anaconda\ana\envs\opencv-env\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0924 09:11:33.836920 24064 deprecation_wrapper.py:119] From D:\Anaconda\ana\envs\opencv-env\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0924 09:11:33.934708 24064 deprecation_wrapper.py:119] From D:\Anaconda\ana\envs\opencv-env\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0924 09:11:34.073341 24064 deprecation_wrapper.py:119] From D:\Anaconda\ana\envs\opencv-env\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0924 09:11:34.174098 24064 deprecation_wrapper.py:

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 20)        1520      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 20)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 50)        25050     
_________________________________________________________________
activation_2 (Activation)    (None, 16, 16, 50)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
__________

### Train and Save Model

In [3]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("model/model-v1.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)
 
earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)
 
#  callback list
callbacks = [earlystop, checkpoint]

model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])
 
nb_train_samples = 1000
nb_validation_samples = 200
epochs = 5
batch_size = 16
 
history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)
 
model.save("model/model-v1.h5")

W0924 09:14:01.575051 24064 deprecation.py:323] From D:\Anaconda\ana\envs\opencv-env\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0924 09:14:01.722655 24064 deprecation_wrapper.py:119] From D:\Anaconda\ana\envs\opencv-env\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/5
62/62 [==============================] - 31s 507ms/step - loss: 1.7777 - acc: 0.3750 - val_loss: 0.1797 - val_acc: 1.0000

Epoch 00001: val_loss improved from inf to 0.17965, saving model to model/model-v1.h5
Epoch 2/5
62/62 [==============================] - 2s 37ms/step - loss: 0.8392 - acc: 0.7147 - val_loss: 0.4901 - val_acc: 0.7969

Epoch 00002: val_loss did not improve from 0.17965
Epoch 3/5
62/62 [==============================] - 2s 31ms/step - loss: 0.4677 - acc: 0.8387 - val_loss: 0.0045 - val_acc: 1.0000

Epoch 00003: val_loss improved from 0.17965 to 0.00449, saving model to model/model-v1.h5
Epoch 4/5
62/62 [==============================] - 2s 29ms/step - loss: 0.4145 - acc: 0.8710 - val_loss: 0.1926 - val_acc: 0.9115

Epoch 00004: val_loss did not improve from 0.00449
Epoch 5/5
62/62 [==============================] - 2s 27ms/step - loss: 0.3355 - acc: 0.8952 - val_loss: 0.0076 - val_acc: 1.0000

Epoch 00005: val_loss did not improve from 0.00449
